In [198]:
import numpy as np
import pandas as pd
from resources import *

In [199]:
R_file = 'data\small_movies_R.csv'
W_file = 'data\small_movies_W.csv'
X_file = 'data\small_movies_X.csv'
Y_file = 'data\small_movies_Y.csv'
b_file = 'data\small_movies_b.csv'


In [200]:
R = import_movies_data(R_file)
b = import_movies_data(b_file)
Y = import_movies_data(Y_file)
movie_list = import_movie_list()
movies = movie_list['title']
movies = np.array(movies).reshape(-1,1)
b = b.reshape(1,-1)
num_movies = R.shape[0]

In [201]:
new_user = np.zeros((Y.shape[0],1))
# say = "continue"
# while say.lower() != "exit" :
#     a = int(input("Enter the Movie ID :  "))
#     while a > Y.shape[0] or a < 0 :
#         a = int(input("Enter the Movie ID which exist int real : "))
    
#     b = float(input(f"Enter the rating you give to {movies[a]} afterwatching it : "))
#     while b > 5 or b < 0: 
#         b = float(input(f"Enter a rating between 0 to 5 ,  it can be decimal too ! : "))

#     new_user[a] = b 
#     say = input("type 'exit' to stop writing the movies otherwise write 'con' to continue typing movies")
new_user = np.zeros((num_movies,1))
new_user[62] = 1
new_user[2716] = 5
new_user[3618] = 1
new_user[1398] = 5
new_user[3083] = 5
new_user[2112] = 3
new_user[2755] = 4
new_user[4170] = 4
new_user[378] = 5
new_user[3773] = 3
new_user[4415] = 4
new_user[3009] = 1
new_user[1088] = 1
new_user[3556] = 5

my_rated = [i for i in range(num_movies) if new_user[i] > 0 ]
for i in range(num_movies):
  if(new_user[i] > 0):
    print(f"User rated {float(new_user[i])} to movie : {movies[i]}")


User rated 1.0 to movie : ['Mission: Impossible II (2000)']
User rated 5.0 to movie : ["Ocean's Eleven (2001)"]
User rated 1.0 to movie : ['Bourne Supremacy, The (2004)']
User rated 5.0 to movie : ['Batman Begins (2005)']
User rated 3.0 to movie : ['Dark Knight, The (2008)']
User rated 5.0 to movie : ['Inception (2010)']
User rated 4.0 to movie : ['Social Network, The (2010)']
User rated 1.0 to movie : ['Moneyball (2011)']
User rated 5.0 to movie : ['Dark Knight Rises, The (2012)']
User rated 5.0 to movie : ['Wolf of Wall Street, The (2013)']
User rated 1.0 to movie : ['Interstellar (2014)']
User rated 3.0 to movie : ['John Wick (2014)']
User rated 4.0 to movie : ['The Intern (2015)']
User rated 4.0 to movie : ['Now You See Me 2 (2016)']


In [202]:
# movies[new_user != 0] , new_user[new_user != 0]
Y = np.c_[new_user , Y]
R = np.c_[(new_user != 0).astype(int), R]
Y.shape , R.shape

((4778, 444), (4778, 444))

In [212]:
Y_norm = mean_normalize_data2(Y , R)
num_users = R.shape[1]
num_genre = 27 

In [204]:
W = np.zeros((Y.shape[1],num_genre))
X = np.zeros((Y.shape[0],num_genre))

In [205]:
def dJ_dw(W,X,R,Y,b,lambda__):
    for i in range(num_users):
        # sum = np.zeros((1,X.shape[1]))
        sum = np.zeros((X.shape[1],1))
        for j in range(num_movies):
            if R[j][i] == 1 :
                wi = np.array(W[i]).reshape(-1,1)
                xj = np.array(X[j]).reshape(-1,1)
                sum = sum + (wi.T.dot(xj) + b[i] - Y[j][i])*xj
        print(W[i].shape , sum.shape)
        W[i] = lambda__*W[i] + sum.T
    return W   

In [206]:
def dJ_dX(W,X,R,Y,b,lambda__):
    for i in range(num_movies):
        sum = np.zeros((1,X.shape[1]))
        for j in range(num_users):
            if R[i][j] == 1  :
                wj = np.array(W[j]).reshape(1,-1)
                xi = np.array(X[i]).reshape(1,-1)
                sum = sum + (wj.dot(xi.T) + b[j] - Y[i][j])*wj 
        X[i] = lambda__*X[i] + sum
    return X


In [207]:
def dJ_db(W,X,R,Y,b):
    for i in range(num_users):
        sum = 0
        for j in range(num_movies):
            if R[j][i] == 1 :
                wi = np.array(W[i]).reshape(1,-1)
                xj = np.array(X[j]).reshape(1,-1)
                sum = sum + wi.dot(xj.T) + b[i] - Y[j][i]
        b[i] = sum
    return b

In [208]:
def cost_fucntion(W,X,Y,R,b,lambda__):
    cost = 0 
    for i in range(num_movies):
        for j in range(num_users):
            if R[i][j] == 1:
                wj = np.array(W[j]).reshape((1,-1))
                xi = np.array(X[i]).reshape((1,-1))
                cost = cost +  (wi.dot(xi) + b[j] - Y[i][j])**2 
    for i in range(num_movies):
        for k in range(num_genre):
            cost = cost +  (lambda__/2)*X[i][k]
        
    for j in range(num_users):
        for k in range(num_genre):
            cost = cost +  (lambda__/2)*W[j][k]

    return cost         
             

In [209]:
iterations = 100
alpha = 1e-5
lambda__ = 1e-3
for i in range(iterations):
    W = W - alpha*dJ_dw(W,X,R,Y,b,lambda__)
    X = X - alpha*dJ_dw(W,X,R,Y,b,lambda__)
    b = b - alpha*dJ_db(W,X,R,Y,b)
    print(f"Cost function value after the iteration {iterations} is : {cost_fucntion(W,X,Y,R,b,lambda__)}")
    

(27,) (27, 443)


ValueError: could not broadcast input array from shape (443,27) into shape (27,)